<a href="https://colab.research.google.com/github/RocketFlash/introduction_to_ML_Innopolis/blob/master/Plates_competition_Keras_siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "rocketflash" 
os.environ['KAGGLE_KEY'] = "980f8b5a341704dc5a289ea502ca31d7"

In [2]:
!kaggle competitions download -c platesv2
!unzip plates.zip

 87% 41.0M/47.1M [00:01<00:00, 19.8MB/s]
100% 47.1M/47.1M [00:01<00:00, 42.3MB/s]
  0% 0.00/8.00k [00:00<?, ?B/s]
100% 8.00k/8.00k [00:00<00:00, 8.55MB/s]
Archive:  plates.zip
   creating: plates/
  inflating: plates/.DS_Store        
   creating: __MACOSX/
   creating: __MACOSX/plates/
  inflating: __MACOSX/plates/._.DS_Store  
   creating: plates/test/
  inflating: plates/test/0071.jpg    
  inflating: plates/test/0717.jpg    
  inflating: plates/test/0703.jpg    
  inflating: plates/test/0065.jpg    
  inflating: plates/test/0059.jpg    
  inflating: plates/test/0515.jpg    
  inflating: plates/test/0273.jpg    
  inflating: plates/test/0267.jpg    
  inflating: plates/test/0501.jpg    
  inflating: plates/test/0529.jpg    
  inflating: plates/test/0298.jpg    
  inflating: plates/test/0461.jpg    
  inflating: plates/test/0307.jpg    
  inflating: plates/test/0313.jpg    
  inflating: plates/test/0475.jpg    
  inflating: plates/test/0449.jpg    
  inflating: plates/test/0105.jpg  

In [3]:
!pip install image-classifiers

     |████████████████████████████████| 81kB 2.9MB/s 


In [4]:
import os
import pandas as pd
import numpy as np
import keras
import shutil
from tqdm import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers

Using TensorFlow backend.


In [0]:
img_size = 224
batch_size = 8

In [6]:
data_root = 'plates/'
dir_for_train = 'to_train/'
os.makedirs(os.path.join(data_root, dir_for_train), exist_ok=True)
train_dir = os.path.join(data_root, dir_for_train,'train')
val_dir = os.path.join(data_root, dir_for_train,'val')
class_names = ['cleaned', 'dirty']
val_each = 6

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
        

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % val_each != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        if file_name.endswith('.jpg'):
            shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|██████████| 21/21 [00:00<00:00, 4639.47it/s]


In [10]:
%%time
train_datagen=ImageDataGenerator(
         rotation_range=360,
         width_shift_range=0.1,
         height_shift_range=0.1,
         shear_range=0.01,
         zoom_range=[0.9, 1.25],
         horizontal_flip=True,
         vertical_flip=True,
         fill_mode='reflect',
         brightness_range=[0.5, 1.5]
        )

train_generator = train_datagen.flow_from_directory(
        'plates/to_train/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary')

val_datagen=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip = True
        )

val_generator = train_datagen.flow_from_directory(
        'plates/to_train/val',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary')


test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(  
        'plates',
        classes=['test'],
        target_size = (img_size, img_size),
        batch_size = 1,
        shuffle = False,        
        class_mode = None)

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 744 images belonging to 1 classes.
CPU times: user 30.2 ms, sys: 5.03 ms, total: 35.2 ms
Wall time: 309 ms


In [0]:
from classification_models import Classifiers

In [12]:
n_classes = 2

# build model
classifier, preprocess_input = Classifiers.get('resnext50')
base_model = classifier(input_shape=(224,224,3), weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(input = base_model.input, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [15]:
%%time

# train
model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=0.0001, decay=1e-6), metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.0000001, verbose=1)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, verbose=1)

callbacks = [reduce_lr, early_stop]

model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        validation_data = val_generator,
        validation_steps = 1,
        epochs=500,
        verbose=1,
        callbacks = callbacks)

Epoch 1/500
200/200 [==============================] - 52s 262ms/step - loss: 0.4822 - acc: 0.8769 - val_loss: 3.7072 - val_acc: 0.5000
Epoch 2/500
200/200 [==============================] - 46s 232ms/step - loss: 0.2860 - acc: 0.9131 - val_loss: 6.8986 - val_acc: 0.5000
Epoch 3/500
200/200 [==============================] - 46s 231ms/step - loss: 0.2778 - acc: 0.9244 - val_loss: 5.7943 - val_acc: 0.6250
Epoch 4/500
200/200 [==============================] - 46s 232ms/step - loss: 0.2259 - acc: 0.9387 - val_loss: 7.4370 - val_acc: 0.5000
Epoch 5/500
200/200 [==============================] - 46s 232ms/step - loss: 0.2137 - acc: 0.9369 - val_loss: 6.0444 - val_acc: 0.6250
Epoch 6/500
200/200 [==============================] - 46s 231ms/step - loss: 0.1643 - acc: 0.9487 - val_loss: 8.2610 - val_acc: 0.3750

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 7/500
200/200 [==============================] - 46s 231ms/step - loss: 0.1112 - acc: 0.9738 - va

In [0]:
%%time
test_generator.reset()
predict = model.predict_generator(test_generator, steps = len(test_generator.filenames))
len(predict)

In [0]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df.head()
sub_df['label'] = predict
sub_df['label'] = sub_df['label'].apply(lambda x: 'dirty' if x > 0.5 else 'cleaned')
sub_df.head()
sub_df['label'].value_counts()
sub_df.to_csv('sub.csv', index=False)

In [0]:
!kaggle competitions submit -c platesv2 -f sub.csv -m 'my second submission'